# Assignment 2

\- Si Nguyen Mai, May 19, 2018 -

In [63]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.feature_selection import SelectFromModel, SelectKBest, RFE

from nltk import ConfusionMatrix
from sklearn.metrics import classification_report

In [64]:
dataset = pd.read_csv('assignment-2_dataset.csv')
print(dataset.shape)
dataset.head()

(6000, 19)


,key,energy,liveliness,tempo,speechiness,acousticness,instrumentalness,time_signature,duration,loudness,valence,danceability,mode,time_signature_confidence,tempo_confidence,key_confidence,mode_confidence,moods,genres
0,8.0,0.831914,0.056928,106.245,0.232162,0.072076,0.000000,1.0,4.0,254.60000,-5.984,0.555338,0.837518,0.555,0.408,0.664,1.000,motivational,rap
1,1.0,0.434896,0.113409,105.276,0.071236,0.456392,0.322574,1.0,4.0,242.21333,-13.902,0.290687,0.459532,0.127,0.225,0.743,0.907,classy,jazz
2,8.0,0.326645,0.843938,123.949,0.492508,0.016599,0.000000,0.0,4.0,275.01714,-10.846,0.568852,0.934398,0.562,0.418,0.704,0.813,sexual,rap
3,7.0,0.873642,0.281888,97.940,0.041479,0.327251,0.264088,0.0,4.0,234.78857,-10.682,0.224093,0.600045,0.000,0.064,0.167,0.983,"funky,mellow",rap
4,5.0,0.117357,0.101048,129.386,0.062007,0.847661,0.010635,0.0,4.0,128.88045,-15.296,0.488700,0.778220,0.369,0.529,0.248,1.000,sprightly,jazz


In [65]:
audio_df = dataset.drop(['moods', 'genres'], axis = 1)
audio_df.head()

,key,energy,liveliness,tempo,speechiness,acousticness,instrumentalness,time_signature,duration,loudness,valence,danceability,mode,time_signature_confidence,tempo_confidence,key_confidence,mode_confidence
0,8.0,0.831914,0.056928,106.245,0.232162,0.072076,0.000000,1.0,4.0,254.60000,-5.984,0.555338,0.837518,0.555,0.408,0.664,1.000
1,1.0,0.434896,0.113409,105.276,0.071236,0.456392,0.322574,1.0,4.0,242.21333,-13.902,0.290687,0.459532,0.127,0.225,0.743,0.907
2,8.0,0.326645,0.843938,123.949,0.492508,0.016599,0.000000,0.0,4.0,275.01714,-10.846,0.568852,0.934398,0.562,0.418,0.704,0.813
3,7.0,0.873642,0.281888,97.940,0.041479,0.327251,0.264088,0.0,4.0,234.78857,-10.682,0.224093,0.600045,0.000,0.064,0.167,0.983
4,5.0,0.117357,0.101048,129.386,0.062007,0.847661,0.010635,0.0,4.0,128.88045,-15.296,0.488700,0.778220,0.369,0.529,0.248,1.000


In [66]:
genres = np.ravel(dataset['genres'])
genres

array(['rap', 'jazz', 'rap', ..., 'rock', 'jazz', 'rap'], dtype=object)

In [67]:
scaler = StandardScaler()
audio_scaled = scaler.fit_transform(audio_df)
audio_scaled

array([[ 0.74289754,  0.79038307, -0.82994424, ..., -0.22933918,
         0.57256136,  0.54444631],
       [-1.20928269, -0.79134335, -0.53280988, ..., -1.25128013,
         0.83133061,  0.07421993],
       [ 0.74289754, -1.22261716,  3.31034567, ..., -0.17349542,
         0.70358377, -0.40106266],
       ..., 
       [-1.48816558, -1.37798984, -0.56033956, ...,  0.98805495,
        -0.66232481,  0.54444631],
       [-0.37263402,  0.04421256, -0.65962109, ..., -1.30712389,
         0.76254385,  0.54444631],
       [ 0.74289754,  0.17007961,  1.7506704 , ..., -0.54764866,
        -0.37079996, -0.2999387 ]])

In [68]:
np.random.seed(777)

In [69]:
X_train, X_test, y_train, y_test = train_test_split(audio_scaled, genres, test_size =  0.3) 

In [70]:
original_classifiers = {'logistic regression': LogisticRegression(solver = 'saga', multi_class = 'multinomial'),
                      'svm': SVC(C = 1, gamma = 1),
                      'rfc': RandomForestClassifier(n_estimators = 5, min_samples_split = 2, max_features = 'log2')}

In [71]:
for name, classifier in original_classifiers.items():
    
    classifier.fit(X_train, y_train)
    prediction = classifier.predict(X_test)
    
    confus_mat = ConfusionMatrix(list(y_test), list(prediction))
    class_report = classification_report(y_test, prediction)
    
    print('Original ' + name)
    print('\n')
    print(confus_mat)
    print(class_report)
    print('\n')

Original logistic regression


      |   d             |
      |   a   j       r |
      |   n   a   r   o |
      |   c   z   a   c |
      |   e   z   p   k |
------+-----------------+
dance |<349> 10  67  46 |
 jazz |  15<393> 14  34 |
  rap |  65  19<328> 17 |
 rock |  38  56  23<326>|
------+-----------------+
(row = reference; col = test)

             precision    recall  f1-score   support

      dance       0.75      0.74      0.74       472
       jazz       0.82      0.86      0.84       456
        rap       0.76      0.76      0.76       429
       rock       0.77      0.74      0.75       443

avg / total       0.77      0.78      0.77      1800



Original svm


      |   d             |
      |   a   j       r |
      |   n   a   r   o |
      |   c   z   a   c |
      |   e   z   p   k |
------+-----------------+
dance |<200>167  41  64 |
 jazz |   6<431>  4  15 |
  rap |  20 158<231> 20 |
 rock |  20 219  13<191>|
------+-----------------+
(row = reference; col = test

In [72]:
param_grids = {
    'logistic regression': 
    {
        'solver': ['newton-cg', 'sag', 'saga', 'lbfgs'], 
        'multi_class': ['ovr', 'multinomial']
    }, 
    'svm': 
    {
        'C': [0.1, 1, 10],
        'gamma': [1, 0.1, 0.01, 0.001]
    },
    'rfc': 
    {
        'n_estimators': [5, 10, 100],
        'min_samples_split': [2, 3, 4, 5, 10],
        'max_features': ['sqrt', 'log2', 'auto']
    }
}

In [90]:
best_classifiers = {'logistic regression': {'estimator': LogisticRegression()},
                    'svm': {'estimator': SVC()},
                    'rfc': {'estimator': RandomForestClassifier()}
                   }

In [91]:
for name, param_grid in param_grids.items():
    
    estimator = best_classifiers[name]['estimator']
    
    grid = GridSearchCV(estimator, param_grid, refit = True, n_jobs = 2)
    grid.fit(X_train, y_train)
    
    best_classifiers[name]['estimator'] = grid.best_estimator_
    best_classifiers[name]['params'] = grid.best_params_
    best_classifiers[name]['prediction'] = grid.best_estimator_.predict(X_test)

In [92]:
best_classifiers

{'logistic regression': {'estimator': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
            intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
            penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
            verbose=0, warm_start=False),
  'params': {'multi_class': 'ovr', 'solver': 'newton-cg'},
  'prediction': array(['dance', 'rap', 'jazz', ..., 'jazz', 'jazz', 'rap'], dtype=object)},
 'rfc': {'estimator': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='sqrt', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=4,
              min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
              oob_score=False, random_state=None, verbose=0,
              warm_start=False),
  'params': {'max_features': 'sqrt',
   'min_samples_split': 4,
   'n_estimators':

In [76]:
for name, classifier in best_classifiers.items():
    
    prediction = classifier['prediction']
    
    confus_mat = ConfusionMatrix(list(y_test), list(prediction))
    class_report = classification_report(y_test, prediction)
    
    print('Best ' + name + ' classifier with parameters: ')
    print(classifier['params'])
    print('\n')
    print(confus_mat, class_report)
    print('\n')

Best logistic regression classifier with parameters: 
{'multi_class': 'ovr', 'solver': 'newton-cg'}


      |   d             |
      |   a   j       r |
      |   n   a   r   o |
      |   c   z   a   c |
      |   e   z   p   k |
------+-----------------+
dance |<340> 10  69  53 |
 jazz |  14<402>  9  31 |
  rap |  63  18<329> 19 |
 rock |  40  61  18<324>|
------+-----------------+
(row = reference; col = test)
              precision    recall  f1-score   support

      dance       0.74      0.72      0.73       472
       jazz       0.82      0.88      0.85       456
        rap       0.77      0.77      0.77       429
       rock       0.76      0.73      0.74       443

avg / total       0.77      0.78      0.77      1800



Best svm classifier with parameters: 
{'C': 1, 'gamma': 0.1}


      |   d             |
      |   a   j       r |
      |   n   a   r   o |
      |   c   z   a   c |
      |   e   z   p   k |
------+-----------------+
dance |<331> 11  75  55 |
 jazz |  13<3

In [93]:
rfc_importances = {'feature': audio_df.columns.values,
                   'importance': best_classifiers['rfc']['estimator'].feature_importances_
                  }
rfc_importances_df = pd.DataFrame(rfc_importances)
rfc_importances_df.sort_values(by = ['importance'], ascending= False, inplace= True)
rfc_importances_df.set_index('feature', inplace = True)
rfc_importances_df

,importance
feature,
acousticness,0.162820
mode,0.132949
speechiness,0.121817
energy,0.088975
tempo,0.080776
key_confidence,0.061952
loudness,0.055894
instrumentalness,0.055109
valence,0.050970


In [95]:
np.mean(best_classifiers['rfc']['estimator'].feature_importances_)

0.058823529411764698

In [96]:
best_classifiers['logistic regression']['estimator'].coef_

array([[-0.05696527,  0.72148895, -0.11871585,  0.35404773, -0.39757135,
        -0.79595924,  0.46994881, -0.25206835,  0.29172998,  0.44311001,
         0.10664753, -0.41165958,  1.15408035, -0.26681219,  0.22492041,
         0.50832284, -0.10801094],
       [-0.20938273, -0.8784298 ,  0.07925582, -0.0823219 , -0.83819243,
         1.46960623,  0.19690176, -0.17440359,  0.14811786,  0.21854349,
        -0.3956161 ,  0.86955807, -0.47283915, -0.1009225 , -0.3960657 ,
        -0.38265274, -0.04658993],
       [ 0.04529943, -0.42494309,  0.09172202, -0.35788793,  1.08122035,
        -0.81640115, -0.3367475 , -0.10335043, -0.18680586, -0.15501524,
         0.48058473,  0.06163259,  0.95507851,  0.06695219, -0.11998802,
        -0.61139229,  0.35680049],
       [ 0.09731279,  0.60691347,  0.0215155 ,  0.06793878, -1.08354152,
        -0.59299192, -0.50553971,  0.4536013 , -0.04099772, -0.41453233,
        -0.23304072,  0.24233002, -1.58152688,  0.16301761,  0.24036245,
         0.15184503

In [97]:
np.mean(best_classifiers['logistic regression']['estimator'].coef_)

-0.021542198127878966

In [98]:
optimized_classifiers = {'logistic regression': {},
                         'svm': {},
                         'rfc': {},
                        }

for key, value in optimized_classifiers.items():
    
    classifier = best_classifiers[key]['estimator']
    
    if key == 'svm':
        value['SKB'] = SelectKBest(k = 6).fit(audio_scaled, genres)
        value['SKB_features'] = audio_df.columns[value['SKB'].get_support(indices = True)].tolist()
        
        skb_X_train = X_train[:, value['SKB'].get_support()].copy()
        skb_X_test = X_test[:, value['SKB'].get_support()].copy()
        
        classifier.fit(skb_X_train, y_train)
        value['SKB_prediction'] = classifier.predict(skb_X_test)
    
    else:
        
        value['SFM'] = SelectFromModel(classifier, prefit = True)
        value['SFM_features'] = audio_df.columns[value['SFM'].get_support(indices = True)].tolist()
        
        sfm_X_train = value['SFM'].transform(X_train.copy())
        sfm_X_test = value['SFM'].transform(X_test.copy())
        
        classifier.fit(sfm_X_train, y_train)
        value['SFM_prediction'] = classifier.predict(sfm_X_test)
        
        value['RFE'] = RFE(classifier, n_features_to_select = 6).fit(X_train, y_train)
        value['RFE_features'] = audio_df.columns.values[value['RFE'].support_]
        value['RFE_prediction'] = value['RFE'].predict(X_test)
                                                        

In [59]:
from sklearn.metrics import accuracy_score    

In [107]:
for key, value in optimized_classifiers.items():
    
    print('\nOptimized ' + key)
    print('\n')
    
    if key == 'svm':
        print('Features selected by SelectKBest: ', value['SKB_features'])
        print('Accuracy of optimized classifier using those features: ',
              accuracy_score(y_test, value['SKB_prediction']))
        print('\n')
    
    else:
        print('Features selected by SelectFromModel: ', value['SFM_features'])
        print("Accuracy of optimized classifier using those features: {}\n".
              format(accuracy_score(y_test, value['SFM_prediction'])))
        
        print('Features selected by RFE: ', value['RFE_features'])
        print('Accuracy of optimized classifier using those features: ', 
              accuracy_score(y_test, value['RFE_prediction']))
        print('\n')
    


Optimized logistic regression


Features selected by SelectFromModel:  ['energy', 'speechiness', 'acousticness', 'danceability', 'mode', 'key_confidence']
Accuracy of optimized classifier using those features: 0.7466666666666667

Features selected by RFE:  ['energy' 'speechiness' 'acousticness' 'danceability' 'mode'
 'key_confidence']
Accuracy of optimized classifier using those features:  0.746666666667



Optimized svm


Features selected by SelectKBest:  ['energy', 'speechiness', 'acousticness', 'valence', 'mode', 'key_confidence']
Accuracy of optimized classifier using those features:  0.746666666667



Optimized rfc


Features selected by SelectFromModel:  ['energy', 'tempo', 'speechiness', 'acousticness', 'mode', 'key_confidence']
Accuracy of optimized classifier using those features: 0.7727777777777778

Features selected by RFE:  ['energy' 'tempo' 'speechiness' 'acousticness' 'loudness' 'mode']
Accuracy of optimized classifier using those features:  0.771666666667




## The effect of normalizing data

In [42]:
X_train_nonscaled, X_test_nonscaled, \
y_train_nonscaled, y_test_nonscaled = train_test_split(audio_df, dataset['genres'], test_size =  0.3)

In [43]:
best_classifiers_nonscaled = {'logistic regression': {'estimator': LogisticRegression()},
                    'svm': {'estimator': SVC()},
                    'rfc': {'estimator': RandomForestClassifier()}
                   }

for name, param_grid in param_grids.items():
    
    estimator = best_classifiers_nonscaled[name]['estimator']
    
    grid = GridSearchCV(estimator, param_grid, refit = True, n_jobs = 2)
    grid.fit(X_train_nonscaled, y_train_nonscaled)
    
    best_classifiers_nonscaled[name]['estimator'] = grid.best_estimator_
    best_classifiers_nonscaled[name]['params'] = grid.best_params_
    best_classifiers_nonscaled[name]['prediction'] = grid.best_estimator_.predict(X_test_nonscaled)
    
for name, classifier in best_classifiers_nonscaled.items():
    
    prediction = classifier['prediction']
    
    confus_mat = ConfusionMatrix(list(y_test_nonscaled), list(prediction))
    class_report = classification_report(y_test_nonscaled, prediction)
    
    print('Best ' + name + ' classifier with parameters: ')
    print(classifier['params'])
    print('\n')
    print(confus_mat, class_report)
    print('\n')

/home/singuyen/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/singuyen/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/singuyen/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/singuyen/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/singuyen/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The ma

Best logistic regression classifier with parameters: 
{'multi_class': 'multinomial', 'solver': 'newton-cg'}


      |   d             |
      |   a   j       r |
      |   n   a   r   o |
      |   c   z   a   c |
      |   e   z   p   k |
------+-----------------+
dance |<338>  7  69  48 |
 jazz |  11<403> 13  45 |
  rap |  54  22<334> 18 |
 rock |  41  56  19<322>|
------+-----------------+
(row = reference; col = test)
              precision    recall  f1-score   support

      dance       0.76      0.73      0.75       462
       jazz       0.83      0.85      0.84       472
        rap       0.77      0.78      0.77       428
       rock       0.74      0.74      0.74       438

avg / total       0.78      0.78      0.78      1800



Best svm classifier with parameters: 
{'C': 10, 'gamma': 0.001}


      |   d             |
      |   a   j       r |
      |   n   a   r   o |
      |   c   z   a   c |
      |   e   z   p   k |
------+-----------------+
dance |<312> 28  66  56 |
 j

In [41]:
selector_param_grids = {
    'SelectKBest': 
    {
        'k': [5, 6, 7, 8, 9] 
    }, 
    'SelectFromModel': 
    {
        'threshold': [-0.5, -0.02, 0.03, 0.05, 0.15, 0.5]
    },
    'RFE': 
    {
        'n_features_to_select': [5, 6, 7, 8, 9]
    }
}